In [1]:
#Import Libraries and Setup Device
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from tqdm import tqdm 
import os
import numpy as np
import random
import time
from torch.optim import lr_scheduler
%env CUDA_LAUNCH_BLOCKING=1

# Setup device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print(f"Using device: {device}")

env: CUDA_LAUNCH_BLOCKING=1
Using device: cuda:0


## customize classifier

In [2]:
# Define the ChineseBERTClassifier Class
# class ChineseBERTClassifier(nn.Module):
#     def __init__(self, multi_terms_backbone, single_term_backbone):
#         super(ChineseBERTClassifier, self).__init__()
#         self.multi_terms_backbone = multi_terms_backbone
#         self.single_terms_backbone = single_term_backbone

#         self.multi_terms_output = nn.Sequential(
#             nn.Linear(768, 1),
#             nn.Sigmoid(dim=1)
#         )
#         self.single_term_output = nn.Sequential(
#             nn.Linear(768, 1),
#             nn.Sigmoid(dim=1)
#         )

#     def forward(self, input_multi, input_single, attention_mask_multi, attention_mask_single):
#         # Main task: get multi-terms score
#         outputs_multi = self.multi_terms_backbone(input_ids=input_multi, attention_mask=attention_mask_multi)
#         multi_embedding = outputs.last_hidden_state[:, 0]  # Get the [CLS] token's embedding
#         multi_score = self.multi_terms_output(multi_embedding)

#         # Aux task: get single term score
#         outputs_single = self.single_term_backbone(input_ids=input_single, attention_mask=attention_mask_single)
#         multi_embedding = outputs.last_hidden_state[:, 0]  # Get the [CLS] token's embedding
        
#         return self.classifier(cls_embedding)

In [23]:

class Relevance_Expert(nn.Module):
    def __init__(self, bert_backbone, num_labels=1):
        super(Relevance_Expert, self).__init__()
        self.bert_backbone = bert_backbone
        # self.proj = nn.Linear(768 * 2, 768)
        self.pre_classifier = nn.Linear(768, 768)
        self.classifier = nn.Sequential(
            nn.Linear(768, num_labels),
            nn.Sigmoid()
        )
        
    def forward(self, input_ids, attention_mask):
         outputs = self.bert_backbone(input_ids=input_ids, attention_mask=attention_mask)
         hidden_state = outputs[0] # (bs, 128, 768)
         embedding = hidden_state[:,0]
         pooled_output = self.pre_classifier(embedding)
         pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
         pred_score = self.classifier(pooled_output)
         return pred_score, embedding

    # def forward(self, q_input_ids, q_attention_mask, a_input_ids, a_attention_mask):
    #     bs = q_input_ids.shape[0]
    #     input_ids = torch.cat([q_input_ids, a_input_ids], dim = 0)
    #     attention_mask = torch.cat([q_attention_mask, a_attention_mask], dim = 0)
    #     outputs = self.bert_backbone(input_ids=input_ids, attention_mask=attention_mask)
    #     hidden_state = outputs[0] # (2 * bs, 128, 768)
    #     q_hidden_state = hidden_state[:bs, :, :]
    #     a_hidden_state = hidden_state[bs:, :, :]
        
    #     q_embedding = q_hidden_state.mean(dim=1)
    #     a_embedding = a_hidden_state.mean(dim=1)
        
    #     embedding = self.proj(torch.cat([q_embedding, a_embedding], dim=1))
        
    #     pred_score = self.classifier(embedding)
    #     return pred_score, embedding


## Create Model and Optimizer

In [24]:
# Define the Evaluation Function

def evaluate(model, data_loader, device):
    model.eval()
    predictions, true_labels = [], []
    thres = 0.6
    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            # outputs,_ = model(q_input_ids=batch['q_input_ids'], q_attention_mask = batch['q_attention_mask'], 
            #                   a_input_ids=batch['a_input_ids'], a_attention_mask = batch['a_attention_mask'])
            outputs,_ = model(input_ids=batch['input_ids'], attention_mask = batch['attention_mask'])
            preds = (outputs.squeeze() > thres).float()
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch['labels'].cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    return accuracy, precision, recall

# def evaluate_regression(model, data_loader, device):
#     model.eval()
#     square_errors = []
#     total_size = 0
#     with torch.no_grad():
#         with tqdm(total=len(data_loader)) as pbar:
#             for batch in data_loader:
#                 batch = {k: v.to(device) for k, v in batch.items()}
#                 outputs, _ = model(input_ids=batch['input_ids'], attention_mask = batch['attention_mask'])
#                 total_size += outputs.shape[0]
#                 square_errors.append(np.sum((outputs.squeeze().cpu().numpy() - batch['labels'].cpu().numpy()) ** 2))
#                 pbar.update(1)
#     mse_score = sum(square_errors) / total_size
#     return mse_score


In [25]:
def train(model, train_dataloader, eval_dataloader, optimizer, loss_fn, save_path, train_args, device='cuda:0',scheduler=None):
    # 开始计时
    start_time = time.time()
    model.train()
    model.to(device)
    total_epochs = train_args['epoch']
    best_acc = 0
    for epoch in range(1, total_epochs + 1):
        model.train()
        for batch in tqdm(train_dataloader, desc = f'epoch [{epoch}/{total_epochs}]'):
            batch = {k: v for k, v in batch.items()}
            # forward
            # outputs,_ = model(q_input_ids=batch['q_input_ids'].to(device), q_attention_mask = batch['q_attention_mask'].to(device), 
            #                   a_input_ids=batch['a_input_ids'].to(device), a_attention_mask = batch['a_attention_mask'].to(device))
            outputs,_ = model(input_ids=batch['input_ids'].to(device), attention_mask = batch['attention_mask'].to(device))
            loss = loss_fn(outputs, batch['labels'].unsqueeze(1).float().to(device))
            
            print(loss)
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if scheduler:
                scheduler.step()
        # evaluate
        print('start evaluating...')
        acc, precision, recall = evaluate(model, eval_dataloader, device)
        # mse = evaluate_regression(model, eval_dataloader, device)
        print(f'evaluate accuracy: {acc}')
        print(f'evaluate precision: {precision}')
        print(f'evaluate recall: {recall}')
        # print(f"evaluate mse: {mse} ")

        # 结束计时
        end_time = time.time()
        # 计算并打印训练时间
        training_time = end_time - start_time
        print(f"Training time: {training_time} seconds")

        # save model
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        if acc > best_acc:
            torch.save(model.state_dict(), os.path.join(save_path, 'best.pth'))
        torch.save(model.state_dict(), os.path.join(save_path, f'epoch_{epoch}.pth'))
        

In [26]:
# Load Data and Prepare Datasets
train_args = {
    'epoch' : 10,
    'batch_size': 128,
    'learning_rate': 4e-5,
    'scheduler': 'None',  # Type of scheduler
    'optimizer': 'adam',
    'loss_fn': 'bce'
}

In [7]:
!export CUDA_LAUNCH_BLOCKING=1

## Load and Preprocess the Dataset

## Compute Weights and Create WeightedRandomSampler

In [8]:
#Define Dataset Class and Helper Functions
# [{'question': '123', 'answer': '456'},{},{}]
# ['123 [SEP] 456', ]
# class dialoguesDataset(Dataset):
#     def __init__(self, q_tokens, a_tokens, labels):
#         self.q_tokens = q_tokens
#         self.a_tokens = a_tokens
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {}
#         item['q_input_ids'] = self.q_tokens['input_ids'][idx].clone().detach()
#         item['q_attention_mask'] = self.q_tokens['attention_mask'][idx].clone().detach()
#         item['a_input_ids'] = self.a_tokens['input_ids'][idx].clone().detach()
#         item['a_attention_mask'] = self.a_tokens['attention_mask'][idx].clone().detach()
#         item['labels'] = self.labels[idx]
#         return item

#     def __len__(self):
#         return len(self.labels)

class dialoguesDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __getitem__(self, idx):
        item = {}
        item['input_ids'] = self.tokens['input_ids'][idx].clone().detach()
        item['attention_mask'] = self.tokens['attention_mask'][idx].clone().detach()
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

def encode_dialogues(tokenizer, dialogues, max_length=128):
    return tokenizer(dialogues, truncation=True, padding='max_length', max_length=max_length, return_tensors="pt")

In [9]:
import random
random.seed(1000)
questions = []
answers = []
dialogues = []
labels = []

with open('extracted_data.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        question, answer = line.split('|')[:2]
        questions.append(question)
        answers.append(answer)
        dialogues.append("[CLS] "+ question.strip() + " [SEP] " + answer.strip())
        labels.append(1)
        
def generate_neg_pairs(dialogues):
    neg_dialogues = []
    # neg_answers = []
    neg_labels = []
    n = len(dialogues)
    for cur_idx, pair in enumerate(dialogues):
        question, ans = pair.split(' [SEP] ')
        neg_idx = random.randint(0, n - 1)
        while neg_idx == cur_idx:
            neg_idx = random.randint(0, n - 1)
        neg_ans = dialogues[neg_idx].split(' [SEP] ')[-1]
        # neg_answers.append(neg_ans)
        neg_dialogues.append(question + " [SEP] " + neg_ans)
        neg_labels.append(0)
    return neg_dialogues, neg_labels
    
def generate_neg_pairs_new(questions, answers):
    neg_answers = []
    neg_labels = []
    n = len(dialogues)
    for cur_idx, q in enumerate(questions):
        question = q
        answer = answers[cur_idx]
        neg_idx = random.randint(0, n - 1)
        while neg_idx == cur_idx:
            neg_idx = random.randint(0, n - 1)
        neg_ans = answers[neg_idx]
        neg_answers.append(neg_ans)
        neg_labels.append(0)
    return neg_answers, neg_labels
    
# neg_answers, neg_labels = generate_neg_pairs_new(questions, answers)
neg_dialogues, neg_labels = generate_neg_pairs(dialogues)
# print(questions[:4])
# print(answers[:4])
# print(neg_answers[:4])
# print(neg_dialogues[:4])
dialogues = [(pos, neg) for pos,neg in zip(dialogues, neg_dialogues)]
labels = [(pos, neg) for pos,neg in zip(labels, neg_labels)]
print(dialogues[:4])
print(labels[:4])



[('[CLS] 是的，我认为狗很听话。你不喜欢吗？ [SEP] 我很喜欢狗,我妈说我连自己都照顾不好,还养狗?', '[CLS] 是的，我认为狗很听话。你不喜欢吗？ [SEP] 我喜欢做的事可多了。'), ('[CLS] 天哪，你喜欢摩托车。你今年几岁？ [SEP] 20岁呀', '[CLS] 天哪，你喜欢摩托车。你今年几岁？ [SEP] 是的啊,挺自由的'), ('[CLS] 你也没有宠物吗？ [SEP] 有啊,你没看到么', '[CLS] 你也没有宠物吗？ [SEP] 北京献过,武汉也献过'), ('[CLS] 哇，太棒了。我也喜欢。我们找个时间出去吧。你开车，我带我的狗，好吗？ [SEP] 你也不会开车?', '[CLS] 哇，太棒了。我也喜欢。我们找个时间出去吧。你开车，我带我的狗，好吗？ [SEP] 没有社团')]
[(1, 0), (1, 0), (1, 0), (1, 0)]


In [10]:
import numpy as np

# Split data into train and test sets
# train_questions, test_questions, train_pos_answers, test_pos_answers, train_neg_answers, test_neg_answers = train_test_split(questions, answers, neg_answers, test_size=0.2)
train_dialogues, test_dialogues, train_labels, test_labels = train_test_split(dialogues, labels, test_size=0.2)
new_train_dialogues = []
new_train_labels = []
new_test_dialogues = []
new_test_labels = []

# new_train_questions = train_questions * 2
# new_test_questions = test_questions * 2
# new_train_answers = train_pos_answers + train_neg_answers
# new_test_answers = test_pos_answers + test_neg_answers
# train_labels = [1] * len(train_pos_answers) + [0] * len(train_neg_answers)
# test_labels = [1] * len(test_pos_answers) + [0] * len(test_neg_answers)

for i, pairs in enumerate(train_dialogues):
    new_train_dialogues.extend(list(pairs))
    new_train_labels.extend(list(train_labels[i]))


for i, pairs in enumerate(test_dialogues):
    new_test_dialogues.extend(list(pairs))
    new_test_labels.extend(list(test_labels[i]))
# print(new_train_questions[:4])
# print(new_train_answers[:4])
print(new_test_labels[:4])


# Encode dialogues using tokenizer
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-bert-wwm-ext")

# train_question_encodings = encode_dialogues(tokenizer, new_train_questions)
# train_answer_encodings = encode_dialogues(tokenizer, new_train_answers)
# test_question_encodings = encode_dialogues(tokenizer, new_test_questions)
# test_answer_encodings = encode_dialogues(tokenizer, new_test_answers)
train_encodings = encode_dialogues(tokenizer, new_train_dialogues)
test_encodings = encode_dialogues(tokenizer, new_test_dialogues)

# Create dataset objects
# train_dataset = dialoguesDataset(q_tokens = train_question_encodings, a_tokens = train_answer_encodings, labels = train_labels)
# test_dataset = dialoguesDataset(q_tokens = test_question_encodings, a_tokens = test_answer_encodings, labels = test_labels)
train_dataset = dialoguesDataset(tokens = train_encodings, labels = new_train_labels)
test_dataset = dialoguesDataset(tokens = test_encodings, labels = new_test_labels)
print(f'train dataset length: {len(train_dataset)}')
print(f'test dataset length: {len(test_dataset)}')
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=train_args['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=train_args['batch_size'], shuffle=False)



[1, 0, 1, 0]
train dataset length: 6276
test dataset length: 1570


In [11]:
test_dataset.__getitem__(2)

{'input_ids': tensor([ 101,  101,  872, 3341, 6821, 2429, 1814, 2356, 3300, 1126,  702, 3299,
          749, 8043,  102, 2769, 1343, 2399, 1282, 3299,  819, 3341, 4638,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0

In [12]:
test_dataset.__getitem__(3)

{'input_ids': tensor([ 101,  101,  872, 3341, 6821, 2429, 1814, 2356, 3300, 1126,  702, 3299,
          749, 8043,  102,  679, 1762, 8024, 2769, 1762, 1266,  776,  511,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0

In [27]:
# Initialize the Model, Optimizer, and Loss Function
from transformers import AutoTokenizer, AutoModelForSequenceClassification,get_linear_schedule_with_warmup

# Load pre-trained model
# model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-bert-wwm-ext",  num_labels=1)
bert_backbone = AutoModel.from_pretrained("hfl/chinese-bert-wwm-ext", return_dict=False)

# Initialize classifier
model = Relevance_Expert(bert_backbone, num_labels=1).to(device)
opt_param = []
for name, param in model.named_parameters():
    print(name)
    opt_param.append(param)

# Setup optimizer and loss function
if train_args['optimizer'] == 'adam':
    optimizer = torch.optim.Adam([{'params': opt_param}], lr=train_args['learning_rate'])
elif train_args['optimizer'] == 'sgd':
    optimizer = torch.optim.SGD([{'params': opt_param}], lr=train_args['learning_rate'])
elif train_args['optimizer'] == 'adamw':
    optimizer = torch.optim.AdamW([{'params': opt_param}], lr=train_args['learning_rate'], eps = 1e-8)



# criterion = nn.CrossEntropyLoss().to(device)
if train_args['loss_fn'] == 'mse':
    criterion = nn.MSELoss().to(device)
elif train_args['loss_fn'] == 'ce':
    criterion = nn.CrossEntropyLoss().to(device)
elif train_args['loss_fn'] == 'bce':
    criterion = nn.BCELoss().to(device)

total_steps = len(train_loader) * train_args["epoch"]
# setup schedualer and learning rate
if train_args["scheduler"] == "linearLR":
    scheduler = lr_scheduler.LinearLR(optimizer, start_factor=0.05, end_factor=0.001, total_iters=30)
elif train_args["scheduler"] == "warmupLR":
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 60, 
                                            num_training_steps = total_steps)
else:
    scheduler = None

print(scheduler)


bert_backbone.embeddings.word_embeddings.weight
bert_backbone.embeddings.position_embeddings.weight
bert_backbone.embeddings.token_type_embeddings.weight
bert_backbone.embeddings.LayerNorm.weight
bert_backbone.embeddings.LayerNorm.bias
bert_backbone.encoder.layer.0.attention.self.query.weight
bert_backbone.encoder.layer.0.attention.self.query.bias
bert_backbone.encoder.layer.0.attention.self.key.weight
bert_backbone.encoder.layer.0.attention.self.key.bias
bert_backbone.encoder.layer.0.attention.self.value.weight
bert_backbone.encoder.layer.0.attention.self.value.bias
bert_backbone.encoder.layer.0.attention.output.dense.weight
bert_backbone.encoder.layer.0.attention.output.dense.bias
bert_backbone.encoder.layer.0.attention.output.LayerNorm.weight
bert_backbone.encoder.layer.0.attention.output.LayerNorm.bias
bert_backbone.encoder.layer.0.intermediate.dense.weight
bert_backbone.encoder.layer.0.intermediate.dense.bias
bert_backbone.encoder.layer.0.output.dense.weight
bert_backbone.encoder.

## training

In [14]:
train_dataset.__getitem__(4)

{'input_ids': tensor([ 101,  101, 2769, 3297, 6818,  671, 4684, 1762, 2110,  739,  511, 2769,
         1920, 6956, 1146, 3198, 7313, 6963, 1762, 5381,  677,  677, 6440,  511,
          872,  679, 1599, 3614, 1220, 4289, 1408, 8043,  102, 2769,  679, 1599,
         3614, 8024, 2769, 1599, 3614, 4318, 4318,  106,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1

In [15]:
criterion

BCEWithLogitsLoss()

In [28]:

train(model = model, 
     train_dataloader= train_loader,
     eval_dataloader= test_loader,
     optimizer = optimizer,
     loss_fn = criterion,
     save_path = 'models',
     train_args = train_args,
     device = device,
     scheduler = scheduler
    )
# evaluate(classifier, test_loader, device = device)

epoch [1/10]:   0%|          | 0/50 [00:00<?, ?it/s]

tensor(0.6963, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


epoch [1/10]:   2%|▏         | 1/50 [00:00<00:29,  1.63it/s]

tensor(0.7162, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


epoch [1/10]:   4%|▍         | 2/50 [00:01<00:28,  1.70it/s]

...

tensor(0.0372, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


epoch [10/10]:  96%|█████████▌| 48/50 [00:27<00:01,  1.75it/s]

tensor(0.0113, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


epoch [10/10]: 100%|██████████| 50/50 [00:28<00:00,  1.78it/s]


tensor(0.0413, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
start evaluating...
evaluate accuracy: 0.9006369426751593
evaluate precision: 0.8784596871239471
evaluate recall: 0.9299363057324841
Training time: 322.9956498146057 seconds
